In [1]:
from dataset import get_datasets


train_dataset, test_dataset = get_datasets()

In [2]:
import pandas as pd
import torch.nn as nn
from torchvision import models

from dataset import PosterDataset


df = pd.read_csv(PosterDataset.csv_file)

resnet = models.resnet50(pretrained=True)
last_layer_classifier = nn.Sequential(
    nn.Linear(resnet.fc.in_features, len(df.columns) - 2), nn.Sigmoid()
)

/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
import wandb

PROJECT_NAME = "STAT4012Project"
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: borisleung2013 (csfyp23). Use `wandb login --relogin` to force relogin


True

In [4]:
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from torch import optim


from model import pretrainedModel


config = {"batch_size": 32, "loss_fn": nn.BCELoss, "optimizer": optim.Adam, "lr": 1e-5}
model = pretrainedModel(
    resnet,
    last_layer_classifier,
    config["loss_fn"],
    config["optimizer"],
    lr=config["lr"],
)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
validation_dataloader = DataLoader(
    test_dataset, batch_size=config["batch_size"], shuffle=False
)

logger = WandbLogger(name="Resnet50-T", project=PROJECT_NAME, log_model="all")

trainer = L.Trainer(max_epochs=10, logger=logger)
trainer.fit(
    model,
    train_dataloaders=validation_dataloader,
    val_dataloaders=validation_dataloader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | features         | Sequential | 23.5 M
1 | class_classifier | Sequential | 38.9 K
2 | loss_fn          | BCELoss    | 0     
------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.188    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/home/boris/anaconda3/envs/csfyp-gpu/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 3431/3431 [1:17:52<00:00,  0.73it/s, v_num=4rv6, train_loss_step=0.0809, val_loss_step=0.0305, val_loss_epoch=0.0613, train_loss_epoch=0.0259]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 3431/3431 [1:17:58<00:00,  0.73it/s, v_num=4rv6, train_loss_step=0.0809, val_loss_step=0.0305, val_loss_epoch=0.0613, train_loss_epoch=0.0259]


In [4]:
import wandb

run = wandb.init()
artifact = run.use_artifact("csfyp23/STAT4012Project/model-h4do4rv6:v9", type="model")
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: borisleung2013 (csfyp23). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-h4do4rv6:v9, 269.91MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.2


In [5]:
import pandas as pd
from torchvision import models
import torch.nn as nn

from dataset import PosterDataset

df = pd.read_csv(PosterDataset.csv_file)
resnet = models.resnet50()
last_layer_classifier = nn.Sequential(
    nn.Linear(resnet.fc.in_features, len(df.columns) - 2), nn.Sigmoid()
)

## Model Evaluation


In [6]:
import os

from model import pretrainedModel

model_loaded = pretrainedModel.load_from_checkpoint(
    os.path.join(artifact_dir, "model.ckpt"),
    pre_trained_model=resnet,
    last_layer_classifier=last_layer_classifier,
)

In [25]:
import random
from dataset import PosterDataset

all_data = PosterDataset()

model_loaded.eval()
random_idx = random.randint(0, len(all_data))
poster, genres = all_data[random_idx]
print(model_loaded(poster.unsqueeze(0)).detach().numpy())
print(genres)

[[0.0048587  0.02325709 0.9205601  0.00957419 0.00118309 0.0078764
  0.02204683 0.13749966 0.01964055 0.00156444 0.00589919 0.02680538
  0.0069188  0.00218627 0.0210513  0.00151185 0.00151907 0.00169181
  0.00676998]]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
# random_idx = random.randint(0, len(all_data))
poster, genres = all_data[1]
print(model_loaded(poster.unsqueeze(0)).detach().numpy())
print(genres)

[[3.0440718e-04 3.2743812e-03 4.3840427e-03 5.9664617e-03 3.0326350e-03
  1.0596698e-03 7.1286243e-01 3.6835000e-03 9.1602273e-02 2.6686905e-02
  7.7897852e-04 4.3404363e-03 1.9517902e-01 1.8097881e-02 1.0290365e-02
  3.2690107e-03 3.9557990e-04 3.9849794e-03 3.3571545e-04]]
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [48]:
from sklearn.metrics import roc_auc_score
import numpy as np
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm


# Assuming `model` is your PyTorch Lightning model
# and `dataloader` is your DataLoader for the validation dataset

model_loaded.eval()  # Set the model to evaluation mode

# Lists to store true labels and prediction probabilities
true_labels = []
pred_probs = []
all_data_loader = DataLoader(all_data, batch_size=1, shuffle=False)

# Disable gradient calculation for evaluation
with torch.no_grad():
    for batch in tqdm(all_data_loader):
        inputs, labels = batch
        # outputs = model_loaded(inputs)
        outputs = model_loaded(inputs.unsqueeze(0))

        # Store true labels and predictions
        true_labels.append(labels)
        pred_probs.append(outputs)

# Concatenate all the batches
true_labels = np.vstack(true_labels)
pred_probs = np.vstack(pred_probs)

# Calculate ROC AUC for each label
roc_auc_scores = {}
for i in range(true_labels.shape[1]):  # Iterate over each label
    roc_auc_scores[f"Label {i}"] = roc_auc_score(true_labels[:, i], pred_probs[:, i])

# Print ROC AUC scores for each label
for label, score in roc_auc_scores.items():
    print(f"{label}: {score:.4f}")

# Calculate the average ROC AUC score across all labels
average_roc_auc = np.mean(list(roc_auc_scores.values()))
print(f"Average ROC AUC: {average_roc_auc:.4f}")

  0%|          | 0/548835 [00:01<?, ?it/s]


KeyboardInterrupt: 